## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bincofre6_0_x = np.load('x_bc6_0.npy')
bincofre6_1_x = np.load('x_bc6_1.npy')
bincofre6_2_x = np.load('x_bc6_2.npy')
bincofre6_3_x = np.load('x_bc6_3.npy')

bincofre6_0_y = np.load('y_bc6_0.npy')
bincofre6_1_y = np.load('y_bc6_1.npy')
bincofre6_2_y = np.load('y_bc6_2.npy')
bincofre6_3_y = np.load('y_bc6_3.npy')

x_bincofre6 = np.concatenate((bincofre6_0_x, bincofre6_1_x, bincofre6_2_x, bincofre6_3_x), axis=0)
y_bincofre6 = np.concatenate((bincofre6_0_y, bincofre6_1_y, bincofre6_2_y, bincofre6_3_y), axis=0)

print(x_bincofre6.shape, y_bincofre6.shape)

(323782, 96, 256) (323782, 96, 1)


In [2]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bincofre6.shape[0])

x_bincofre6 = x_bincofre6[p]
y_bincofre6 = y_bincofre6[p]

print(x_bincofre6.shape, y_bincofre6.shape)

(323782, 96, 256) (323782, 96, 1)


## (7) 모델

In [3]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,96, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(144, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [4]:

model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bincofre6,
           y_bincofre6,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()

print('걸린시간:', endtime - starttime)
model1.save('gcc6_bin_core_s96_h144_o0123.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 96, 288)           461952    
_________________________________________________________________
time_distributed (TimeDistri (None, 96, 1)             289       
Total params: 462,241
Trainable params: 462,241
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 226647 samples, validate on 97135 samples
Epoch 1/500
226647/226647 [==============================] - 132s 583us/sample - loss: 0.0033 - accuracy: 0.9991 - val_loss: 7.3370e-04 - val_accuracy: 0.9998
Epoch 2/500
226647/226647 [==============================] - 126s 556us/sample - loss: 5.6154e-04 - accuracy: 0.9998 - val_loss: 5.0829e-04 - val_ac

걸린시간: 2539.4397237300873
save 완료
